# DC_2019

## Main foreign keys: 
INCIDENT_ID\
VICTIM_ID\
OFFENDER_ID

## Vocabulary
- LEA:   Law Enforcement Agency
- LEOKA: Law Enforcement Officer Killed or Assaulted
- Crime Incident ([source](https://www.icpsr.umich.edu/web/pages/NACJD/NIBRS/concepts.html "concepts_source"))
    - Central to NIBRS is the concept of a crime incident. An "incident" is defined for NIBRS reporting purposes as one or more offenses committed by the same offender, or group of offenders acting in concert, at the same time and place. "Acting in concert" requires that the offenders actually commit or assist in the commission of the crime(s). **The offenders must be aware of, and consent to, the commission of the crime(s); or even if nonconsenting, their actions assist in the commission of the offense(s).** This is important because all of the offenders in an incident are considered to have committed all of the offenses in the incident. If one or more of the offenders did not act in concert, then there is more than one incident involved.
        "Same time and place" means that the time interval between the offenses and the distance between the locations where they occurred were insignificant. Normally, the offenses must have occurred during an unbroken time duration and at the same or adjoining location(s). However, incidents can also be comprised of offenses which by their nature involve continuing criminal activity by the same offender(s) at different times and places, as long as the activity is deemed to constitute a single criminal transaction. Examples of crime incidents are included in the NIBRS codebook appendices.

## Tables exploration

### Using offenders instead of arrestees
Incidents:
- Shape: (4105, 15)
- Unique: 4105

Offenders:
- Shape: (5720, 11)
- Unique: 5720

Arrestees:
- Shape: (2175, 15)
- Unique: 2175

All incidents IDs from the Incidents table can be found in the offenders table. 

Also, Group B offenses are reported with only the arrest information but I checked the group for all the OFFENSE_ID linked to the INCIDENT_ID from the NIBRS_OFFENSE.csv and they are all in group A.

So I will use Offenders since it has enough information for our purpose and we are more interested in the offender than in the arrest.

### NIBRS_INCIDENT
- Months have internal IDs (might be unique per "agency"). The actual year and "normal" month number can be found by joining with the NIBRS_month table
- CARGO_THEFT_FLAG indicates if the incident involved a cargo theft. This is reported due to the economic impact of these thefts
- INCIDENT_DATE is indeed the date or the beginning of the time-period in which the incident occurred.
- REPORT_DATE_FLAG: if equals R, the date of the incident is unknown and the date is the report date.
- CLEARED_EXCEPT_ID column may have the following reasons:
    - Death of Offender
    - Prosecution Declined
    - In Custody of Other Jurisdiction
    - Victim Refused to Cooperate
    - Juvenile/No Custody
    - Not Applicable
- Not interesting:
    - DATA_HOME: no staging, always "C"
    - FF_LINE_NUMBER: unused
    - DID: seems to be internal classification. Seems not relevant for us.
- **JOIN_fields**:
    - with **NIBRS_month** on NIBRS_MONTH_ID to get month and year
    - with **NIBRS_VICTIM** on VICTIM_ID to get the VICTIM_ID
    - with **NIBRS_OFFENDER** on OFFENDER_ID to get the OFFENDER_ID
    - with **NIBRS_OFFENSE** on INCIDENT_ID to get the OFFENSE_ID

### NIBRS_OFFENDER
- An incident may have several offenders, to each is given a sequential number per incident with OFFENDER_SEQ_NUM
- Out of 7 AGE_ID values (0 to 6) there are only values 4 and 5, corresponding respectively "Unknown" and age in years
- When AGE_ID is 5:
    - AGE_NUM: 
        - will be either exact age or the average age if the value range column is populated. However:
            - if one value in the AGE_RANGE_HIGH_NUM column is 0 (AGE_RANGE_LOW_NUM column has no values at 0), the value taken is not the average but the value in the AGE_RANGE_LOW_NUM column
            - there are not AGE_RANGE_LOW_NUM with 0 value, only AGE_RANGE_HIGH_NUM has 0 values
- Before 2016, not possible to know if AGE_NUM is average or exact age
- **JOIN_fields**:
    - with **NIBRS_VICTIM_OFFENDER_REL** on OFFENDER_ID to get the RELATIONSHIP_ID

### NIBRS_VICTIM
- An incident may have several victims, to each is given a sequential number per incident with VICTIM_SEQ_NUM
- VICTIM_TYPE_ID, used to categorize the victim. Categories should be like "Government, Institution, Individual, etc..." but there are numbers in the table and not reference to what those numbers mean.
- RESIDENT_STATUS_CODE: the victim was a resident in the reporting agency's jurisdiction at the time of the incident. R = Resident; N = Nonresident
- **JOIN_fields**:
    - with **NIBRS_VICTIM_OFFENDER_REL** on VICTIM_ID to get the RELATIONSHIP_ID
    - with **NIBRS_VICTIM_CIRCUMSTANCES** on VICTIM_ID to get CIRCUMSTANCES_ID
    - with **NIBRS_VICTIM_TYPE** on VICTIM_TYPE_ID to get type of victim (maybe only individuals)

### NIBRS_OFFENSE
- OFFENSE_ID: Internal unique ID.
- OFFENSE_TYPE_ID: references the type of offense. Should be joinded with the NIBRS_OFFENSE_TYPE table to get the offense description
- LOCATION_ID: joined with the NIBRS_LOCATION_TYPE table gives the kind of location. Interesting for geographical data
- **SplitFields:** 
    - ATTEMPT_COMPLETE_FLAG: Whether the offense was attempted or completed. A = Attempted; C = Completed. Could be used for a split in an analysis.
    - METHOD_ENTRY_CODE: Method of entry of a structure when offense is Burglary. F = Force; N = No Force. Could be used for a split but maybe more about methods or to look for correlation between No Force and victime-offender relationship
- **JOIN fields**:
    - with **NIBRS_OFFENSE_TYPE** on OFFENSE_TYPE_ID to get the offense description
    - with **NIBRS_BIAS_MOTIVATION** on OFFENSE_ID to get the BIAS_ID
    - with **NIBRS_OFFENSE_TYPE** on OFFENSE_TYPE_ID to get OFFENSE_NAME and OFFENSE_CATEGORY_NAME
    - with **NIBRS_LOCATION_TYPE** on LOCATION_ID to get geographical location type

### NIBRS_OFFENSE_TYPE
- OFFENSE_CODE can be left out, and use OFFENSE_TYPE_ID with OFFENSE_NAME instead
- **GroupingFields:**
    - OFFENSE_CATEGORY_NAME already provides categories for the offenses
- **SplitFields:** 
    - CRIME_AGAINST can be used to split between crime against person or property
    - OFFENSE_GROUP would make a split between A and B groups offenses. Only interesting if we want, for instance, to include group B offenses since those have only arrest information

### NIBRS_BIAS_LIST / NIBRS_BIAS_MOTIVATION
- **JOIN fields**:
    - with **NIBRS_BIAS_MOTIVATION** on BIAS_ID to get the motivation description based on ethnicity

### NIBRS_RELATIONSHIP / NIBRS_VICTIM_OFFENDER_REL 
- RELATIONSHIP_TYPE_ID is divided in four levels:
    - 1: blood relative or close **family**
    - 2: not familry but victim knew offender
    - 3: victim did not know offender
    - 4: victime is offender
- **JOIN fields**:
    - with **NIBRS_VICTIM_OFFENDER_REL** on RELATIONSHIP_TYPE_ID to get the relationship between victim and offender

### NIBRS_CIRCUMSTANCES / NIBRS_VICTIM_CIRCUMSTANCES
- **JOIN fields**:
    - with **NIBRS_VICTIM_CIRCUMSTANCES** on CIRCUMSTANCES_ID to get circumstances of offense

## Used tables:
- agencies???
- NIBRS_BIAS_LIST
- NIBRS_BIAS_MOTIVATION
- NIBRS_CIRCUMSTANCES
- NIBRS_INCIDENT
- NIBRS_LOCATION_TYPE
- NIBRS_month
- NIBRS_OFFENDER
- NIBRS_OFFENSE_TYPE
- NIBRS_OFFENSE
- NIBRS_RELATIONSHIP
- NIBRS_VICTIM_CIRCUMSTANCES
- NIBRS_VICTIM_OFFENDER_REL
- NIBRS_VICTIM_TYPE
- NIBRS_VICTIM
- REF_STATE???


## Discarded tables:
- NIBRS_ACTIVITY_TYPE
- NIBRS_AGE (not needed as ages have only values 4 and 5)
- NIBRS_ARREST_TYPE (out of scope)
- NIBRS_ARRESTEE_WEAPON (out of scope)
- NIBRS_ARRESTEE (will use offenders instead of arrestees)
- NIBRS_ASSIGNMENT_TYPE (out of scope)
- NIBRS_CLEARED_EXCEPT (out of scope)
- NIBRS_CRIMINAL_ACT_TYPE (out of scope, no need for juridic qualification of act)
- NIBRS_CRIMINAL_ACT (out of scope, no need for juridic qualification of act)
- NIBRS_DRUG_MEASURE_TYPE (out of scope)
- NIBRS_ETHNICITY (left out by choice)
- NIBRS_INJURY (out of scope)
- NIBRS_JUSTIFIABLE_FORCE (out of scope)
- NIBRS_PROP_DESC_TYPE (out of scope)
- NIBRS_PROP_LOSS_TYPE
- NIBRS_PROPERTY_DESC (out of scope)
- NIBRS_PROPERTY (out of scope)
- NIBRS_SUSPECT_USING (out of scope)
- NIBRS_SUSPECTED_DRUG_TYPE
- NIBRS_SUSPECTED_DRUG
- NIBRS_USING_LIST
- NIBRS_VICTIM_INJURY (out of scope)
- NIBRS_VICTIM_OFFENSE (can get info with other joins)
- NIBRS_WEAPON_TYPE
- NIBRS_WEAPON (out of scope)
- REF_RACE (choice)
- NIBRS_VICTIM_INJURY (out of scope)

## Working notes


- the 1833 observations of the "AGE_ID" column (for the victim) for which the value is NaN correspond to the observations of the "SEX_CODE" column (for the victim) for which the value is NaN
- confirmed that when AGE_RANGE_HIGH_NUM column is 0, the value taken is not the average but the value in the AGE_RANGE_LOW_NUM column
- according to the NIBRS guide: *"When age range is populated, the system will store the average age here, decimal value truncated."*\ However, AGE_RANGE_HIGH_NUM and AGE_RANGE_LOW_NUM are always populated (except when "SEX_CODE" is not populated; which might then indicate that the value is missing alltogether). Only AGE_RANGE_HIGH_NUM can have its value set to 0 which tends to indicate that "not populated" means "value set to 0". If this logic is true, this means that we have 3234 records for which we know the age of the victim (i.e. records where "AGE_RANGE_HIGH_NUM" is set to 0 and the "AGE_ID" is set to 5)

- plan for missing values:
    - INCIDENT_HOUR: fill with mean
    - offender_gender data: drop it, as we'll concentrate on the victim's POV
    - victim data: drop columns with NaN values (as this does not change the NaN values in the Relationship data)
    - CIRCUMSTANCES columns: drop them, only about 310 values there
    - RELATIONSHIP columns...?